In [ ]:
using FundamentalsNumericalComputation

# Chapter 1

## Floating-point numbers

This [Section](https://tobydriscoll.net/fnc-julia/intro/floating-point.html) was primarily about issues that arise working with floating point numbers.  The main takeaways were:

* Floating point numbers are approximations of real numbers. The consist of zero and all numbers of the form:

$$
\pm (1 + f) \times 2^n
$$

where $n$ is the exponent, and $1+f$ is the significand:

$$
f = \sum_{i=1}^d \frac{b_i}{2^i}, ~~~~~ b_i \in \{0,1\}
$$


* Machine epsilon $\epsilon_{\text{mach}}$ is the smallest number that can be added to 1 and still be different from 1.

* Double precision (IEEE 754) floating point numbers have 53 bits of precision, and so have $\epsilon_{\text{mach}} = 2^{-52} \approx 2.2 \times 10^{-16}$.

* IEEE754 also includes NaN, -Inf and Inf. These are useful for handling errors and overflows.

### Exercises 1.1

#### 1.1.1 

Consider a floating point set with $d=4$.

a) How many elements of F are there in the interval $[1/2,4]$ including the endpoints?

Between each power of 2 there are 2^4 elements (including the first end point). So there are 2^4*3 + 1  = 49 elements in the interval.  

b) What is element of F that is closest to $1/10$?

The closest element must have n = -4 (as  2^-4 = 1/16 < 1/10 < 1/8 = 2^-3).  
So we want (1+f)/16 = 1/10, or f = 0.6.  The hint suggests just enumerating all the candidates.  Since `0b0.1001` is .5625, and `0b0.1010` is .625, we see that the closest is `0.1010' or f=0.625. so that the final answers is 1.625*2^-4 =0.1015625.  

c) What is the smallest positive integer not in F? 
Here we need to look for the value of the exponent for which the spacing between floating point numbers is greater than 1. The spacing is 2^-4*2^n. So we need to find the smallest n such that 2^-4*2^n > 1. This is n = 5. So the smallest positive integer not in F is 32+1=33. We can verify this  in Julia by using the floating point hex representation of floating point numbers. In this case we can write `0x1.0p5` which is 32, and then the next larger integer is `0x1.1p5` which is 34.  So 33 is not in F. 
Note that for n=4 the spacing is exactly 1, so the elements of F between 16 and 32 are exactly integers.

#### 1.1.5

In [2]:
nextfloat(floatmax())

Inf

In [3]:
nextfloat(-Inf64)

-1.7976931348623157e308

Thats interesting. This say that Inf and -Inf sit 'one $\epsilon$ away from the maximum values.

## Problems and conditioning

This section discussed subtractive cancellation as a main source of error in numerical computations. It also introduced the concept of conditioning of a problem, which is a measure of how sensitive the solution is to small changes in the input data:

Condition Number:

$$
\begin{align}
\kappa &= \frac{\text{relative change in output}}{\text{relative change in input}} \\
       &= \frac{\frac{\Delta f}{f}}{\frac{\Delta x}{x}} \\
       &= \frac{|f(x)- f(x + \epsilon x)| }{|\epsilon f(x)| }
\end{align}
$$

If $f$ is differentiable:

$$
\kappa = \frac{|xf'(x)|}{|f(x)|}
$$

For example, the condition number of the function $f(x) = x + c$ is $\frac{|x|}{|x + c|}$, which is large when $x$ is close to $-c$.  This is is the subtractive cancellation issue.  

The condition number can be used to approximate the relative error in the output given the relative error in the input.

For composite functions, the condition numbers muiltply. If $h(x) = f(g(x))$, then:

$$
\kappa_h(x) = \kappa_f (g(x)) \times \kappa_g(x)
$$  

## Algorithms

This section was primarily an introduction to writing algorithms as code in Julia. 

### Exercises

#### 1.3.1
Write a julia function `poly1p` that evaluates a polynomial `p`.  at x = -1. You should do this directly.

In [41]:
function poly1(p)
    n = length(p)
    res = 0.0
    for i in 1:n
        res += p[i]*(-1)^(i-1)
    end
    res
end

poly1([1,-1,3])

5.0

In [42]:
poly1([0,-1,2])

3.0

#### 1.3.2
Compute sample variance

In [43]:
function samplevar(x)
    n = length(x)
    sum((x .- mean(x)).^2) / (n-1)
end

samplevar (generic function with 1 method)

In [47]:
using Statistics
test = rand(200)
var(test) - samplevar(test)

-2.7755575615628914e-17

In [49]:
eps(var(test))

1.3877787807814457e-17

I don't know if it is cheating to use the built in sum and mean and broadcasting ?

## Stability

This section defines an algorithm as "unstable" if the error exceeds what conditioning can explain. The example they give is of using the standard quadradic formula to find the roots of a quadradic polynomial. The standard formula are unstable because of the subtraction of two nearly equal numbers. However if instead you find the 'stable' root and then use the identity $x_1x_2 = c/a$ to find the other root, then you can avoid the subtraction and the algorithm is stable.

This section also defines 'backward error' as the error in the input that would cause the computed result. This is a useful concept because it allows you to determine if the error is due to the algorithm or the input. If the backward error is small, then the algorithm is stable. If the backward error is large, then the algorithm is unstable.





### Exercises

#### Exercise 1.4.2

Let $f(x) = \frac{e^x -1}{x}$

a) find the condition number of $f$.  What is the maximum over $-1 \leq x \leq 1$?

$$
f'(x) = \frac{e^x}{x} - \frac{e^x - 1}{x^2} = \frac{e^x(x-1) + 1}{x^2}
$$

So the condition number is:

$$
\kappa = \frac{e^x(x-1) + 1}{e^x-1} = x - 1 + \frac{x}{e^x-1}
$$

The maximum is at $x=1$ and is about 0.58

In [56]:
x = [10.0^-i for i in 2:11]
obvious = (exp.(x) .- 1) ./ x

10-element Vector{Float64}:
 1.0050167084167947
 1.0005001667083846
 1.000050001667141
 1.000005000006965
 1.0000004999621837
 1.0000000494336803
 0.999999993922529
 1.000000082740371
 1.000000082740371
 1.000000082740371

In [64]:
x

10-element Vector{Float64}:
 0.010000000000000002
 0.001
 0.0001
 1.0e-5
 1.0e-6
 1.0000000000000001e-7
 1.0e-8
 1.0e-9
 1.0e-10
 1.0e-11

In [66]:
coefficients = [1/factorial(i) for i in 1:8]
f_mac(x) = Polynomial(coefficients)(x)
series = [f_mac(xi) for xi in x]

10-element Vector{Float64}:
 1.0050167084168058
 1.0005001667083417
 1.0000500016667084
 1.0000050000166667
 1.0000005000001666
 1.0000000500000017
 1.000000005
 1.0000000005
 1.00000000005
 1.000000000005

In [67]:
(series.-obvious)./obvious

10-element Vector{Float64}:
  1.1046811613451617e-14
 -4.283318501737127e-14
 -4.325212636097066e-13
  9.70174641418975e-12
  3.798293112844667e-11
  5.663214061968672e-10
  1.1077471007857804e-8
 -8.224036415312194e-8
 -8.269036415312195e-8
 -8.273536415312195e-8

So the relative error increases... so what? We expect the series approximation to become more accurate with smaller x, so the increased relative error is not due to that. Is likely due instead to the increase in subtractive cancellation that occurs as exp(x) becomes closer to 1 (even though the condition number is decreasing!) this means the obvious algorthm for computing $(e^x-1)/x$ is unstable near 0.